In [ ]:
from markitdown import MarkItDown
import os 
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter 
from pypdf import PdfReader, PdfWriter

Garder la dernière page du PDF : 

In [12]:
def extract_last_page(input_path, output_path):
    # Read the original PDF
    reader = PdfReader(input_path)
    writer = PdfWriter()

    # Get the last page
    last_page = reader.pages[-1]
    writer.add_page(last_page)

    # Write it to a new PDF
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

    print(f"Last page saved to: {output_path}")

In [13]:
def is_html(file_path):
    with open(file_path, "rb") as f:
        header = f.read(1024).lower()
        return b"<html" in header or b"<!doctype html" in header

In [ ]:
path = 'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs' # chemin d'accès des pdfs
subdirectories = os.listdir(path)

for fichier in subdirectories:
    if not is_html(f'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs\\{fichier}'):
        myFile = f'pdfs\{fichier}'
        extract_last_page(myFile, f'{myFile}_extract_')

Conversion PDF en Markdown

In [6]:
markitdown = MarkItDown()
result = markitdown.convert('pdfs\ACSA%201_2.pdf')
print(result.text_content)

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\700113543.py:2: SyntaxWarning: invalid escape sequence '\A'
  result = markitdown.convert('pdfs\ACSA%201_2.pdf')


ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

Busan

Yokohama

Busan

Yokohama

Shanghai
Ningbo

Shanghai
Ningbo

Keelung

Chiwan

Hong Kong

Manzanillo

Chancay
Callao

Arica

Iquique

INCA

Chancay
Callao

San Antonio
Lirquen

INCA2

Balboa

Posorja

Chancay
Callao
Callao

San Antonio

CMA CGM Strengths

•  Direct and unique product offering fastest transit time between Far East & West Coast South America (WCSA)
•  Direct call to San Antonio, offering the most competitive transit time on the market
•  Seamless connection with CMA CGM direct fixed-day weekly INCA services (to connect Arica/Iquique/Lirquen)
•  Designed for perishable and fresh fruit in reefer containers from West Coast South America to the Far East

ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

TRANSIT TIMES*

EASTBOUND

SAN ANTONIO

CHANCAY

CALLAO

POSORJA

YOKOHAMA

BUSAN

SHANGHAI

NINGBO

41

36

32

29

WESTBOUND

49

45

4